In [8]:
from lsst.daf.butler import Butler
from astropy.table import Table
import pylab
import numpy
from astropy.stats import mad_std
from tqdm import tqdm
from astropy.io import fits
import pandas as pd
from scipy import stats
from multiprocessing import Pool
import pickle
import os

In [9]:
repo_path = "embargo_new"
butler = Butler(repo_path,instrument='LSSTCam')
registry = butler.registry

We query what collections we have in the repository

In [10]:
#13590
#13535
#13525
# https://confluence.slac.stanford.edu/pages/viewpage.action?pageId=405094202
collections = list(registry.queryCollections("*stability*E1496*"))
collections

['u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35/20241031T044355Z',
 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z',
 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35/20241103T150501Z',
 'u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35',
 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35',
 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35']

We query what dataset are in this collections

In [11]:
sensor="R23_S00"
detector=list(registry.queryDimensionRecords("detector",instrument='LSSTCam',where=f"detector.full_name='{sensor}'"))[0].id
detector

99

In [12]:
import traceback
def getcoef( detector ):
    print(collections,detector)
    df=butler.get('flat_gain_stability_stats',collections=collections,detector=detector)

    expids=set(list(df['exposure']))

    mjd=[]
    temps1=[]
    temps2=[]
    expid=[]
    ccdtemp=[]
    atempu=[]
    led1temp=[]
    led2temp=[]    
    for ref in tqdm(sorted( expids )[::]):
        try:
            header = butler.get('raw.metadata',
                                dataId={"exposure": ref},
                                collections=collections,detector=detector)
            expid.append(ref)
            temps1.append(header['TEMP6'])
            temps2.append(header['TEMP10'])
            mjd.append(header['MJD'])
            ccdtemp.append(header['CCDTEMP'])
            atempu.append(header['ATEMPU'])
            led1temp.append(header['TEMPLED1'])
            led2temp.append(header['TEMPLED2'])
            
        except:
            traceback.print_exc()
            pass

    dfmerged=pd.merge(df, pd.DataFrame({'exposure': expid, 'TEMP6': temps1, 'TEMP10': temps2,
                                        'MJD': mjd, 'CCDTEMP': ccdtemp, 'ATEMPU': atempu,
                                        'LED1TEMP': led1temp, 'LED2TEMP': led2temp,
                                       }), on='exposure', how='inner')

    ret = {}
    for amp in list(set(dfmerged.amp_name)):
        subset=dfmerged[dfmerged['amp_name']==amp]
        y0=(subset['median']/subset['pd_integral']).min()
        x=(subset['TEMP6']+subset['TEMP10'])/2
        y=subset['median']/subset['pd_integral']/y0
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        ret.update( {amp: { 
            "mjd": subset['MJD'],
            "ccdtemp": subset['CCDTEMP'],
            "ATEMPU": subset['CCDTEMP'],            
            "x": x,
            "y": y,
            "y0": y0,
            "slope": slope,
            "intercept": intercept,
            "r_value": r_value,
            "p_value": p_value,
            "std_err": std_err } })
    return ret

In [ ]:
for did in list(registry.queryDimensionRecords("detector",instrument='LSSTCam'))[:189]:
    print(did.id, did.full_name)
    path = f'/home/y/youtsumi/notebooks/Run7/gaintemp_E1496/{did.full_name}.pickle'
    print(path)
    if os.path.exists(path):
        continue

    ret={ did.full_name: getcoef(did.id) }
    with open(path,'wb') as f:
        pickle.dump(ret,f)
    
#getcoef('')

0 R01_S00
/home/y/youtsumi/notebooks/Run7/gaintemp_E1496/R01_S00.pickle
['u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35/20241031T044355Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35/20241103T150501Z', 'u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35'] 0


100%|██████████| 555/555 [00:33<00:00, 16.71it/s]


1 R01_S01
/home/y/youtsumi/notebooks/Run7/gaintemp_E1496/R01_S01.pickle
['u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35/20241031T044355Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35/20241103T150501Z', 'u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35'] 1


100%|██████████| 555/555 [00:32<00:00, 16.82it/s]


2 R01_S02
/home/y/youtsumi/notebooks/Run7/gaintemp_E1496/R01_S02.pickle
['u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35/20241031T044355Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35/20241103T150501Z', 'u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35'] 2


...

100%|██████████| 555/555 [00:32<00:00, 16.89it/s]


31 R10_S11
/home/y/youtsumi/notebooks/Run7/gaintemp_E1496/R10_S11.pickle
['u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35/20241031T044355Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35/20241103T150501Z', 'u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35'] 31


100%|██████████| 555/555 [00:33<00:00, 16.76it/s]


157 R34_S11
/home/y/youtsumi/notebooks/Run7/gaintemp_E1496/R34_S11.pickle
['u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35/20241031T044355Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35/20241103T150501Z', 'u/lsstccs/eo_flat_gain_stability_pd_baseline_fix_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35', 'u/lsstccs/eo_flat_gain_stability_pd_clippedmean_baseline_fix_E1496_w_2024_35'] 157


 42%|████▏     | 234/555 [00:13<00:18, 17.05it/s]

In [ ]:
header